# Writing *n*-dimensional Code with ML4Science

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tum-pbs/ML4Science/blob/main/docs/N_Dimensional.ipynb)
&nbsp; • &nbsp; [🌐 **ML4Science**](https://github.com/tum-pbs/ML4Science)
&nbsp; • &nbsp; [📖 **Documentation**](https://tum-pbs.github.io/ML4Science/)
&nbsp; • &nbsp; [🔗 **API**](https://tum-pbs.github.io/ML4Science/ml4s)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/tum-pbs/ML4Science/blob/main/docs/Examples.ipynb) [**Examples**](https://tum-pbs.github.io/ML4Science/Examples.html)

ML4Science's [dimension types](Shapes.html) allow you to write abstract code that scales with the number of *spatial* dimensions.

In [15]:
%%capture
!pip install ml4s

from ml4s import math
from ml4s.math import spatial, channel, instance

## Grids

Grids are a popular data structure that in *n* dimensions.
In ML4Science, each axis of the grid is represented by a spatial dimension.

In [2]:
grid_1d = math.random_uniform(spatial(x=5))
grid_2d = math.random_uniform(spatial(x=3, y=3))
grid_3d = math.random_uniform(spatial(x=16, y=16, z=16))

Note that the dimension names are arbitrary.
We chose `x`, `y`, `z` for readability.

Now, let's write a function that outputs the mean of the direct neighbors of each cell.
In 1D, this would be the stencil (.5, 0, .5) and in 2D (0, .25, 0; .25, 0, .25; 0, .25, 0).

In [3]:
def neighbor_mean(grid):
    left, right = math.shift(grid, (-1, 1), padding=math.extrapolation.PERIODIC)
    return math.mean([left, right], math.non_spatial)

This function uses [`math.shift()`](ml4s/math#ml4s.math.shift) to access the left and right neighbor in each direction.
By default, `shift` shifts in all spatial dimensions and lists the result along a new channel dimension.
Then we can take the mean of the `right` and the `left` values to compute the mean of all neighbors.

We can now evaluate the function in 1D, 2D, 3D, etc. and it will automatically derive the correct stencil.

In [4]:
neighbor_mean(grid_1d)

(0.569, 0.954, 0.517, 0.759, 0.679) along xˢ

In [5]:
neighbor_mean(grid_2d)

(xˢ=3, yˢ=3) 0.384 ± 0.085 (3e-01...5e-01)

In [6]:
neighbor_mean(grid_3d)

(xˢ=16, yˢ=16, zˢ=16) 0.502 ± 0.116 (1e-01...9e-01)

To make sure that the stencil is correct, we can look at the [matrix representation](Matrices.html) of our function.

In [7]:
math.print(math.matrix_from_function(neighbor_mean, grid_1d)[0])

x=0     0.   0.5  0.   0.   0.5  along ~x
x=1     0.5  0.   0.5  0.   0.   along ~x
x=2     0.   0.5  0.   0.5  0.   along ~x
x=3     0.   0.   0.5  0.   0.5  along ~x
x=4     0.5  0.   0.   0.5  0.   along ~x


In [8]:
math.print(math.matrix_from_function(neighbor_mean, grid_2d)[0])

x&y=0     0.    0.25  0.25  0.25  0.    0.    0.25  0.    0.    along ~x&~y
x&y=1     0.25  0.    0.25  0.    0.25  0.    0.    0.25  0.    along ~x&~y
x&y=2     0.25  0.25  0.    0.    0.    0.25  0.    0.    0.25  along ~x&~y
x&y=3     0.25  0.    0.    0.    0.25  0.25  0.25  0.    0.    along ~x&~y
x&y=4     0.    0.25  0.    0.25  0.    0.25  0.    0.25  0.    along ~x&~y
x&y=5     0.    0.    0.25  0.25  0.25  0.    0.    0.    0.25  along ~x&~y
x&y=6     0.25  0.    0.    0.25  0.    0.    0.    0.25  0.25  along ~x&~y
x&y=7     0.    0.25  0.    0.    0.25  0.    0.25  0.    0.25  along ~x&~y
x&y=8     0.    0.    0.25  0.    0.    0.25  0.25  0.25  0.    along ~x&~y


The same principle holds for all grid functions in the `ml4s.math` library.
For example, if we perform a Fourier transform, the algorithm will be selected based on the number of spatial dimensions.
A 1D FFT will always be performed on our 1D grid, even if we add additional non-spatial dimensions.

In [11]:
math.fft(grid_1d)  # 1D FFT

((3.4769058+0j), (0.13400637-0.38783944j), (0.44312537+0.4848501j), (0.44312537-0.4848501j), (0.13400637+0.38783944j)) along xˢ complex64

In [12]:
math.fft(grid_2d)  # 2D FFT

(xˢ=3, yˢ=3) complex64 |...| < 3.4559051990509033

In [13]:
math.fft(grid_3d)  # 3D FFT

(xˢ=16, yˢ=16, zˢ=16) complex64 |...| < 2057.66064453125

## Dimensions as Components

Not all applications involving physical space use grids to represent data.
Take point clouds or particles for instance.
In these cases, we would represent the dimensionality not by the number of spatial dimensions but by the number of vector components.

In [17]:
points_1d = math.random_uniform(instance(points=4), channel(vector='x'))
points_2d = math.random_uniform(instance(points=4), channel(vector='x,y'))
points_3d = math.random_uniform(instance(points=4), channel(vector='x,y,z'))

In these cases, the generalization to *n* dimensions is usually trivial.
Take the following function that computes the pairwise distances.

In [22]:
def pairwise_distances(x):
    return math.vec_length(math.rename_dims(x, 'points', 'others') - x)

Here, we compute the distances between each pair of particles on a matrix with dimensions `points` and `others`.
The intermediate matrix of position distances inherits the vector dimension from `x` and [`math.vec_length()`](ml4s/math#ml4s.math.vec_length) sums all components.
Consequently, this function computes the correct distances in 1D, 2D and 3D.

In [23]:
pairwise_distances(points_1d)

(othersⁱ=4, pointsⁱ=4) 0.219 ± 0.179 (0e+00...5e-01)

In [24]:
pairwise_distances(points_2d)

(othersⁱ=4, pointsⁱ=4) 0.330 ± 0.219 (0e+00...6e-01)

In [25]:
pairwise_distances(points_3d)

(othersⁱ=4, pointsⁱ=4) 0.375 ± 0.281 (0e+00...8e-01)

## Further Reading

Here, we focussed on spatial dimensions, but each [dimension type](Shapes.html) plays a unique role in ML4Science.

The library [Φ<sub>Flow</sub>](https://github.com/tum-pbs/PhiFlow) uses ML4Science to implement an *n*-dimensional incompressible fluid solver.

[🌐 **ML4Science**](https://github.com/tum-pbs/ML4Science)
&nbsp; • &nbsp; [📖 **Documentation**](https://tum-pbs.github.io/ML4Science/)
&nbsp; • &nbsp; [🔗 **API**](https://tum-pbs.github.io/ML4Science/ml4s)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/tum-pbs/ML4Science/blob/main/docs/Examples.ipynb) [**Examples**](https://tum-pbs.github.io/ML4Science/Examples.html)